In [ ]:
from transformers import AutoImageProcessor, BeitModel
import os
import pandas as pd
from PIL import Image
import torch
from transformers import BeitImageProcessor

********Biet**

In [ ]:
src_dir = '/kaggle/input/image-search/test/images'
query_dir = '/kaggle/input/image-search/queries/queries'
submission = pd.read_csv('/kaggle/input/image-search/sample_submission.csv')
model = BeitModel.from_pretrained("Raghavan/beit3_base_patch16_480_vqa").cuda().eval()
processor = BeitImageProcessor.from_pretrained("Raghavan/beit3_base_patch16_480_vqa")

In [ ]:
submission['dot_class'] = 22
submission['cosine_class'] = 22
with torch.no_grad():
    query_images = []
    query_classes = []
    for file in os.listdir(query_dir):
        inputs = processor(images=[Image.open(os.path.join(query_dir, file)).convert('RGB')], return_tensors='pt').to('cuda')
        outputs = model.forward(inputs.pixel_values)
        outputs = outputs.pooler_output
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        query_images.append(outputs)
        query_classes.append(int(file[:-4]))
    query_images = torch.cat(query_images)
    for idx, row in submission.iterrows():
        if not pd.isna(row['class']):
            continue
        inputs = processor(images=[Image.open(os.path.join(src_dir, row['img_file'])).convert('RGB')], return_tensors='pt').to('cuda')
        outputs = outputs = model.forward(inputs.pixel_values)
        outputs = outputs.pooler_output
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        values = outputs @ query_images.T
        if values.softmax(1).max() > .05:
            submission.at[idx, 'dot_class'] = query_classes[values.cpu().argmax().numpy().tolist()]
        cosine = torch.cosine_similarity(outputs, query_images)
        if cosine.max() > 0.8:
            submission.at[idx, 'cosine_class'] = query_classes[cosine.cpu().argmax().numpy().tolist()]

    sub = submission[['img_file',]]
    sub['class'] = submission['dot_class']
    sub.to_csv('dot_product.csv', index=False)
    sub['class'] = submission['cosine_class']
    sub.to_csv('cosine_similarity.csv', index=False)

SigLip

In [ ]:
from transformers import (
    AutoModel,
    CLIPProcessor,AutoProcessor
)
import os
import pandas as pd
from PIL import Image
import torch


src_dir = '/kaggle/input/image-search/test/images'
query_dir = '/kaggle/input/quries/queries'
submission = pd.read_csv('/kaggle/input/image-search/sample_submission.csv')
model = AutoModel.from_pretrained("google/siglip-so400m-patch14-384").cuda().eval()
processor = AutoProcessor.from_pretrained("google/siglip-so400m-patch14-384")

submission['dot_class'] = 22
submission['cosine_class'] = 22
with torch.no_grad():
    query_images = []
    query_classes = []
    for file in os.listdir(query_dir):
        inputs = processor(images=[Image.open(os.path.join(query_dir, file)).convert('RGB')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cpu()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        query_images.append(outputs)
        query_classes.append(int(file[:-4]))
    query_images = torch.cat(query_images)
    for idx, row in submission.iterrows():
        if not pd.isna(row['class']):
            continue
        inputs = processor(images=[Image.open(os.path.join(src_dir, row['img_file'])).convert('RGB')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cpu()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        values = outputs @ query_images.T
        if values.softmax(1).max() > .05:
            submission.at[idx, 'dot_class'] = query_classes[values.argmax().numpy().tolist()]
        cosine = torch.cosine_similarity(outputs, query_images)
        if cosine.max() > 0.75:
            submission.at[idx, 'cosine_class'] = query_classes[cosine.argmax().numpy().tolist()]

    sub = submission[['img_file',]]
    sub['class'] = submission['dot_class']
    sub.to_csv('dot_product_seg2.csv', index=False)
    sub['class'] = submission['cosine_class']
    sub.to_csv('cosine_similarity_seg2.csv', index=False)

# Try Code

In [ ]:
from transformers import (
    AutoModel,
    CLIPProcessor,AutoProcessor
)
import os
import pandas as pd
from PIL import Image
import torch

In [ ]:
src_dir = '/kaggle/input/image-search/test/images/'
query_dir = '/kaggle/input/image-search/queries/queries'
submission = pd.read_csv('/kaggle/input/image-search/sample_submission.csv')
model = AutoModel.from_pretrained("google/siglip-so400m-patch14-384").cuda().eval()
processor = AutoProcessor.from_pretrained("google/siglip-so400m-patch14-384")

In [ ]:
from PIL import Image
file = "599a1cad-7536-4150-b502-cb5720a4b82e.jpg"
processor(images=[Image.open(os.path.join(src_dir, file)).convert('RGB')], return_tensors='pt').to('cuda')
outputs = model.get_image_features(inputs.pixel_values).cpu()
outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
values = outputs @ query_images.T
cosine = torch.cosine_similarity(outputs, query_images)


In [ ]:
outputs.shape

In [ ]:
values

In [ ]:
cosine

In [ ]:
cosine.max()

In [ ]:
cosine.argmax().numpy().tolist()

In [ ]:
if cosine.max() > 0.8:
    submission.at[idx, 'cosine_class'] = query_classes[cosine.argmax().numpy().tolist()]

In [ ]:
from IPython.display import Image

# Replace 'path/to/your/image.jpg' with the actual path to your image file
image_path = '/kaggle/input/image-search/test/images/e4f64bb3-e7d0-4cee-9846-1ed53324a4a5.jpg'

# Display the image
Image(filename=image_path)


# Preprocess query

In [ ]:
!pip install super-image

# Try to Improve queries 

In [ ]:
from PIL import Image
import torch
from torchvision.transforms import ToTensor
from super_image import EdsrModel, ImageLoader

# Replace 'path/to/your/image.jpg' with the actual path to your image file
image_path = '/kaggle/input/image-search/queries/queries/18.jpg'

# Load the image using PIL
image = Image.open(image_path).convert('RGB')

# Transform the image to tensor
input_tensor = ToTensor()(image).unsqueeze(0)

# Load the EDSR model
model = EdsrModel.from_pretrained('eugenesiow/edsr', scale=4)  # Replace with the desired scale

# Make predictions
with torch.no_grad():
    preds = model(input_tensor)

# Display the original and upscaled images (optional)
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt

to_pil = ToPILImage()
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(to_pil(input_tensor.squeeze(0)))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Upscaled Image')
plt.imshow(to_pil(preds.squeeze(0)))
plt.axis('off')

plt.show()



In [ ]:
import csv
from collections import Counter

def count_labels(csv_file_path, label_column):
    # Initialize a Counter to count occurrences of each label
    label_counter = Counter()

    # Read the CSV file
    with open(csv_file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        
        # Iterate through rows and count labels
        for row in reader:
            label = row[label_column]
            label_counter[label] += 1

    # Print the label counts
    for label, count in label_counter.items():
        print(f'{label}: {count}')

# Example usage
csv_file_path = '/kaggle/working/cosine_similarity.csv'
label_column_name = 'class'  # Replace with the actual column name containing class labels
count_labels(csv_file_path, label_column_name)


In [ ]:
count_labels("/kaggle/working/cosine_similarity1.csv", "class")

In [ ]:
import cv2
import numpy as np

# Read the image
image = cv2.imread('/kaggle/input/image-search/queries/queries/0.jpg')

# Define a kernel for morphological operations
kernel = np.ones((3, 3), np.uint8)

# Apply closing operation

image_dialte = cv2.dilate(image,kernel)
image_erode = cv2.erode(image,kernel)
closing_result = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
open_result = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# Display the original and the result
plt.figure(figsize=(20, 5))

# Plot each image in a separate subplot
plt.subplot(1, 5, 1)
plt.imshow(image)
plt.axis('off')  # Turn off axis labels if you don't need them

plt.subplot(1, 5, 2)
plt.imshow(image_dialte)
plt.axis('off')

plt.subplot(1, 5, 3)
plt.imshow(image_erode)
plt.axis('off')

plt.subplot(1, 5, 4)
plt.imshow(closing_result)
plt.axis('off')

plt.subplot(1, 5, 5)
plt.imshow(open_result)
plt.axis('off')

# Adjust layout for better spacing
plt.tight_layout()

# Show the figure
plt.show()

In [ ]:
open_result = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
#plt.imshow(image)
# plt.imshow(image_dialte)
# plt.imshow(image_erode)
#plt.imshow(closing_result)
plt.imshow(open_result)
plt.show()

In [ ]:
import os

# Specify the path for the new folder
new_folder_path = '/kaggle/working/queries/'

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)


In [ ]:
import cv2
import os

# Read the old picture file
folder = '/kaggle/input/image-search/queries/queries/'
file_name = '4.jpg'
old_picture = cv2.imread(folder + file_name)

# Specify the folder where you want to save the new picture
output_folder = '/kaggle/working/queries/'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save the modified picture to the specified folder
cv2.imwrite(os.path.join(output_folder, file_name ), old_picture)


In [ ]:
os.remove("/kaggle/working/queries/new_picture.jpg")

In [ ]:
count_labels("/kaggle/working/cosine_similarity1.csv", "class")

# SAM (Segmentation )

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
image = cv2.imread('/kaggle/input/image-search/queries/queries/4.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
!pip -q install datasets open_clip_torch

In [ ]:
# load clip
#"google/siglip-so400m-patch14-384"
def load_clip(model_name: str = "ViT-L-14-336",
              pretrained_name: str = "openai"):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  model, _, preprocess = open_clip.create_model_and_transforms(model_name,
                                                               device = device,
                                                               pretrained = pretrained_name)
  # model.load_state_dict(torch.load(pretrained_name,
  #                                  map_location = "cpu")["state_dict"])
  return model.to(device), preprocess

In [ ]:
import os
import urllib
from functools import lru_cache
from random import randint
from typing import Any, Callable, Dict, List, Tuple

import open_clip
import cv2
import numpy as np
import PIL
import torch
import matplotlib.pyplot as plt
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

In [ ]:
CHECKPOINT_PATH = "sam_vit_h_4b8939.pth"
CHECKPOINT_URL = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"
MODEL_TYPE = "default"
MAX_WIDTH = MAX_HEIGHT = 400
THRESHOLD = 0.05
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def adjust_image_size(image: np.ndarray) -> np.ndarray:
    height, width = image.shape[:2]
    MAX_WIDTH = MAX_HEIGHT = 400

    if height > width:
        if height > MAX_HEIGHT:
              height, width = MAX_HEIGHT, int(MAX_HEIGHT / height * width)
    else:
        if width > MAX_WIDTH:
            height, width = int(MAX_WIDTH / width * height), MAX_WIDTH
    image = cv2.resize(image, (width, height))
    return image

@torch.no_grad()
def get_similarity_score(crop_images: List[PIL.Image.Image], prompt: str) -> torch.Tensor:
    from open_clip import tokenizer

    model, preprocess = load_clip()
    image_input = [preprocess(crop) for crop in crop_images]
    image_input = torch.stack(image_input).to(device)
    text_input = tokenizer.tokenize(prompt).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        text_features = model.encode_text(text_input).float()

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (image_features @ text_features.T)
    similarity = similarity.detach().cpu().numpy()
    print("max: ", np.max(similarity), "ave: ", np.mean(similarity))
    return similarity

def filter_masks(image: np.ndarray, masks: List[Dict[str, Any]],
                 predicted_iou_threshold: float, stability_score_threshold: float,
                 prompt: str, clip_threshold: float) -> List[Dict[str, Any]]:

    cropped_masks: List[PIL.Image.Image] = []
    filtered_masks: List[Dict[str, Any]] = []

    for mask in masks:
        if (mask["predicted_iou"] < predicted_iou_threshold) or (mask["stability_score"] < stability_score_threshold):
            continue
        filtered_masks.append(mask)

        # crop image from mask
        x, y, w, h = mask["bbox"]
        masked = image * np.expand_dims(mask["segmentation"], -1)
        crop = masked[y: y + h, x: x + w]
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        crop = PIL.Image.fromarray(crop)
        cropped_masks.append(crop)

        if prompt and filtered_masks:
            most_similar_masks = []
            scores = get_similarity_score(cropped_masks, prompt)
            for i, score in enumerate(scores):
                if score > clip_threshold:
                      most_similar_masks.append(filtered_masks[i])

    return most_similar_masks

def draw_masks(image: np.ndarray, masks: List[np.ndarray],
               alpha: float = 0.7) -> np.ndarray:
    for mask in masks:
        color = [randint(127, 255) for _ in range(3)]

        # draw mask
        colored_mask = np.expand_dims(mask["segmentation"], 0).repeat(3, axis=0)
        colored_mask = np.moveaxis(colored_mask, 0, -1)
        masked = np.ma.MaskedArray(image, mask=colored_mask, fill_value=color)
        image_overlay = masked.filled()
        image = cv2.addWeighted(image, 1 - alpha, image_overlay, alpha, 0)

        # draw contour
        contours, _ = cv2.findContours(
            np.uint8(mask["segmentation"]), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        cv2.drawContours(image, contours, -1, (255, 0, 0), 1)

    return image

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
masks = mask_generator.generate(image)

In [ ]:
masks

In [ ]:
image = adjust_image_size(image)

In [ ]:
predicted_iou_threshold = 0.5
stability_score_threshold = 0.8
clip_threshold = 0.24
prompt = "Logo"

In [ ]:
filter_mask = filter_masks(
      image,
      masks,
      predicted_iou_threshold,
      stability_score_threshold,
      prompt,
      clip_threshold,
  )

In [ ]:
image_draw = draw_masks(image,filter_mask )
image_draw = cv2.cvtColor(image_draw, cv2.COLOR_BGR2RGB)
plt.imshow(image_draw)
plt.show()

In [ ]:
import cv2
import numpy as np

# Load the original image and segmentation mask
original_image = cv2.imread('/kaggle/input/image-search/queries/queries/0.jpg')
segmentation_mask = cv2.cvtColor(image_draw, cv2.COLOR_BGR2GRAY)


# Ensure the mask is binary (0 or 255)
_, segmentation_mask = cv2.threshold(segmentation_mask, 127, 255, cv2.THRESH_BINARY)

# Invert the mask
inverse_mask = cv2.bitwise_not(segmentation_mask)

# Apply the inverse mask to get the regions not segmented
result_image = cv2.bitwise_and(original_image, original_image, mask=inverse_mask)

# Save or display the result
plt.imshow(result_image)
plt.show()


In [ ]:
image_draw = image_draw

# Convert the colored image to grayscale for segmentation mask
segmentation_mask = cv2.cvtColor(image_draw, cv2.COLOR_BGR2GRAY)

# Ensure the mask is binary (0 or 255)
_, segmentation_mask = cv2.threshold(segmentation_mask, 127, 255, cv2.THRESH_BINARY)

# Invert the mask
inverse_mask = cv2.bitwise_not(segmentation_mask)

# Assuming you have the original image loaded as well (original_image)
original_image = cv2.imread('/kaggle/input/image-search/queries/queries/0.jpg')

# Apply the inverse mask to get the regions not segmented
result_image = cv2.bitwise_and(original_image, original_image, mask=inverse_mask)
plt.imshow(result_image)
plt.show()

# MASKS

In [ ]:
image = cv2.imread('/kaggle/input/image-search/queries/queries/11.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
masks = mask_generator.generate(image)

In [ ]:
all_mask_image = draw_masks(image,masks)
plt.imshow(all_mask_image)
plt.show()

In [ ]:
import os
import cv2


# Specify the input and output folders
input_folder = '/kaggle/input/image-search/queries/queries/'
output_folder = 'edit_queries/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # Add more file extensions if needed
        # Load the original image
        original_image = cv2.imread(os.path.join(input_folder, filename))
        image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        masks = mask_generator.generate(image)
        all_mask_image = draw_masks(image,masks)
        
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, all_mask_image)

print("Processing complete.")


In [ ]:
from transformers import (
    AutoModel,
    CLIPProcessor,AutoProcessor
)
import os
import pandas as pd
from PIL import Image
import torch


src_dir = '/kaggle/input/image-search/test/images'
query_dir = '/kaggle/working/edit_queries'
submission = pd.read_csv('/kaggle/input/image-search/sample_submission.csv')
model = AutoModel.from_pretrained("google/siglip-so400m-patch14-384").cuda().eval()
processor = AutoProcessor.from_pretrained("google/siglip-so400m-patch14-384")

submission['dot_class'] = 22
submission['cosine_class'] = 22
with torch.no_grad():
    query_images = []
    query_classes = []
    for file in os.listdir(query_dir):
        inputs = processor(images=[Image.open(os.path.join(query_dir, file)).convert('RGB')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cpu()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        query_images.append(outputs)
        query_classes.append(int(file[:-4]))
    query_images = torch.cat(query_images)
    for idx, row in submission.iterrows():
        if not pd.isna(row['class']):
            continue
        inputs = processor(images=[Image.open(os.path.join(src_dir, row['img_file'])).convert('RGB')], return_tensors='pt').to('cuda')
        outputs = model.get_image_features(inputs.pixel_values).cpu()
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        values = outputs @ query_images.T
        if values.softmax(1).max() > .05:
            submission.at[idx, 'dot_class'] = query_classes[values.argmax().numpy().tolist()]
        cosine = torch.cosine_similarity(outputs, query_images)
        if cosine.max() > 0.75:
            submission.at[idx, 'cosine_class'] = query_classes[cosine.argmax().numpy().tolist()]

    sub = submission[['img_file',]]
    sub['class'] = submission['dot_class']
    sub.to_csv('dot_product_seg.csv', index=False)
    sub['class'] = submission['cosine_class']
    sub.to_csv('cosine_similarity_seg.csv', index=False)

In [ ]:
alpha = 0.7
color = [randint(127, 255) for _ in range(3)]

# draw mask
colored_mask = np.expand_dims(mask["segmentation"], 0).repeat(3, axis=0)
colored_mask = np.moveaxis(colored_mask, 0, -1)
masked = np.ma.MaskedArray(image, mask=colored_mask, fill_value=color)
image_overlay = masked.filled()
image = cv2.addWeighted(image, 1 - alpha, image_overlay, alpha, 0)

# draw contour
contours, _ = cv2.findContours(
        np.uint8(mask["segmentation"]), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (255, 0, 0), 1)

plt.imshow(image)
plt.show()

In [ ]:
adjust_image = adjust_image_size(image)
plt.imshow(adjust_image)
plt.show()

In [ ]:
print(len(masks))
print(masks[0].keys())

In [ ]:
area = []
iou = []
for i in range(len(masks)):       
    area += [masks[i]['area']]
    iou += [masks[i]['predicted_iou']]

    
mask =  masks[iou.index(max(ioou))]

In [ ]:

a = masks[iou.index(max(iou))]

In [ ]:
# Extract mask pixel coordinates
mask_pixel_coords = np.column_stack(np.where(a['segmentation']))

# Create an empty image
mask_image = Image.new('1', (a['bbox'][2], a['bbox'][3]), 0)

# Set pixels in the mask
for coord in mask_pixel_coords:
    mask_image.putpixel((coord[1], coord[0]), 1)

# Save or display the mask image

mask_image.show()

In [ ]:
masks[area.index(max(area))]

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show() 

In [ ]:
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

In [ ]:
masks2 = mask_generator_2.generate(image)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks2)
plt.axis('off')
plt.show() 

# CLIPSeg

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
from PIL import Image

image = Image.open("/kaggle/input/image-search/queries/queries/0.jpg")

In [ ]:
from transformers import CLIPSegProcessor, CLIPSegForImageSegmentation

processor = CLIPSegProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegForImageSegmentation.from_pretrained("CIDAS/clipseg-rd64-refined")

In [ ]:

prompts = ["LogoBrand","Outer thing"]

inputs = processor(text=prompts, images=[image] * len(prompts), padding="max_length", return_tensors="pt")

In [ ]:

import torch
import matplotlib.pyplot as plt

# predict
with torch.no_grad():
  outputs = model(**inputs)

preds = outputs.logits.unsqueeze(1)

# visualize prediction
_, ax = plt.subplots(1, 5, figsize=(15, 4))
[a.axis('off') for a in ax.flatten()]
ax[0].imshow(image)
[ax[i+1].imshow(torch.sigmoid(preds[i][0])) for i in range(4)];
[ax[i+1].text(0, -15, prompts[i]) for i in range(4)];

# Instance Segmentation (SAM) 
 

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def mask_im(mask,  random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    return mask_image
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    

In [ ]:
image = cv2.imread('/kaggle/input/image-search/queries/queries/14.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
original = image

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.axis('on')
plt.show()

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
predictor.set_image(image)

In [ ]:
input_point = np.array([[140, 115]])
input_label = np.array([1])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)

In [ ]:
scores.max()

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()  

In [ ]:
input_point = np.array([[140, 115] ,[220,115]]) 
input_label = np.array([1,1])
mask_input = logits[np.argmax(scores), :, :]  # Choose the model's best mask


In [ ]:
masks, _, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
show_mask(masks, plt.gca())
show_points(input_point, input_label, plt.gca())
plt.axis('off')
plt.show() 

In [ ]:
mask_image = mask_im(masks)
mask_image = mask_image[:,:,:3]


img_n = cv2.normalize(src=mask_image, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [ ]:
gray = cv2.cvtColor(img_n , cv2.COLOR_RGB2GRAY)
plt.imshow(gray, cmap='gray')
plt.show()

In [ ]:
binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

In [ ]:
bw = cv2.bitwise_and(original,original, mask=binary_image)

In [ ]:
plt.imshow(bw)
plt.show()

In [ ]:
os.makedirs("edit_alone/")

In [ ]:
path = "/kaggle/working/edit_alone/"
cv2.imwrite(output_path+ "1.jpg", bw)

In [ ]:
img = cv2.imread("/kaggle/working/edit_alone/0.jpg")

# Make Folder

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

    #sam_vit_b_01ec64.pth
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
import sys

sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
def make_mask_bw(path_file):
    image = cv2.imread(path_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original = image
    predictor.set_image(image)
    input_point = np.array([[image.shape[0]//2, image.shape[1]//2]])
    input_label = np.array([1])

    masks, scores, logits = predictor.predict(
      point_coords=input_point,
      point_labels=input_label,
      multimask_output=True,
    )

    for i, (mask, score) in enumerate(zip(masks, scores)):
        if abs(score - scores.max()) <= 10**-16:
            mask_image = mask_im(mask)

    mask_image = mask_image[:,:,:3]
    img_n = cv2.normalize(src=mask_image, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    gray = cv2.cvtColor(img_n , cv2.COLOR_RGB2GRAY)
    binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
    bw = cv2.bitwise_and(original,original, mask=binary_image)
    return bw

In [ ]:
import os
import cv2


# Specify the input and output folders
input_folder = '/kaggle/input/image-search/queries/queries/'
output_folder = 'edit_queries/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # Add more file extensions if needed
        # Load the original image
        bw = make_mask_bw(input_folder + filename)
        
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, bw)

print("Processing complete.")

In [ ]:
folder = "/kaggle/working/edit_queries/"
for i in os.listdir(folder):
    im = cv2.imread(folder + i)
    plt.imshow(im)
    plt.show()